In [ ]:
# ==========================================
# UIDAI HACKATHON - COMPLETE ANALYSIS
# Based on your exact column structure
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
import os

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("="*70)
print(" "*20 + "UIDAI HACKATHON")
print(" "*18 + "COMPLETE ANALYSIS")
print("="*70)

# ==========================================
# SETUP PATHS
# ==========================================

BASE_PATH = os.path.abspath('..')
DATA_PATH = os.path.join(BASE_PATH, 'data', 'processed')
OUTPUT_PATH = os.path.join(BASE_PATH, 'outputs', 'figures')

# Create output folder
os.makedirs(OUTPUT_PATH, exist_ok=True)

# ==========================================
# LOAD DATA
# ==========================================

print("\n LOADING DATA...")

df_enrollment = pd.read_csv(os.path.join(DATA_PATH, 'enrollment_combined.csv'))
df_demographic = pd.read_csv(os.path.join(DATA_PATH, 'demographic_combined.csv'))
df_biometric = pd.read_csv(os.path.join(DATA_PATH, 'biometric_combined.csv'))

print(f" Enrollment: {len(df_enrollment):,} rows")
print(f" Demographic: {len(df_demographic):,} rows")
print(f" Biometric: {len(df_biometric):,} rows")

# ==========================================
# DATA CLEANING & PREPARATION
# ==========================================

print("\n CLEANING DATA...")

# Convert dates
df_enrollment['date'] = pd.to_datetime(df_enrollment['date'], errors='coerce')
df_demographic['date'] = pd.to_datetime(df_demographic['date'], errors='coerce')
df_biometric['date'] = pd.to_datetime(df_biometric['date'], errors='coerce')

# Remove rows with invalid dates
df_enrollment = df_enrollment.dropna(subset=['date'])
df_demographic = df_demographic.dropna(subset=['date'])
df_biometric = df_biometric.dropna(subset=['date'])

# Create total enrollment column
df_enrollment['total_enrollments'] = (
    df_enrollment['age_0_5'].fillna(0) + 
    df_enrollment['age_5_17'].fillna(0) + 
    df_enrollment['age_18_greater'].fillna(0)
)

# Create total demographic updates column
df_demographic['total_updates'] = (
    df_demographic['demo_age_5_17'].fillna(0) + 
    df_demographic['demo_age_17_'].fillna(0)
)

# Create total biometric updates column
df_biometric['total_updates'] = (
    df_biometric['bio_age_5_17'].fillna(0) + 
    df_biometric['bio_age_17_'].fillna(0)
)

# Create time features
for df in [df_enrollment, df_demographic, df_biometric]:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    df['day_of_week'] = df['date'].dt.dayofweek

# Handle state names (clean up)
df_enrollment['state'] = df_enrollment['state'].str.strip().str.title()
df_demographic['state'] = df_demographic['state'].str.strip().str.title()
df_biometric['state'] = df_biometric['state'].str.strip().str.title()

# Remove duplicates
df_enrollment = df_enrollment.drop_duplicates()
df_demographic = df_demographic.drop_duplicates()
df_biometric = df_biometric.drop_duplicates()

print(f" Cleaned enrollment: {len(df_enrollment):,} rows")
print(f" Date range: {df_enrollment['date'].min()} to {df_enrollment['date'].max()}")
print(f" Total enrollments: {df_enrollment['total_enrollments'].sum():,.0f}")
print(f" States: {df_enrollment['state'].nunique()}")
print(f" Districts: {df_enrollment['district'].nunique()}")

# ==========================================
# ANALYSIS 1: TIME SERIES TREND
# ==========================================

print("\n" + "="*70)
print(" ANALYSIS 1: TIME SERIES TREND")
print("="*70)

# Monthly aggregation
monthly = df_enrollment.groupby(df_enrollment['date'].dt.to_period('M'))['total_enrollments'].sum()
monthly.index = monthly.index.to_timestamp()

# Plot
fig, ax = plt.subplots(figsize=(16, 7))
ax.plot(monthly.index, monthly.values, marker='o', linewidth=2.5, markersize=5, color='#2E86AB')
ax.fill_between(monthly.index, monthly.values, alpha=0.3, color='#2E86AB')
ax.set_title('Monthly Enrollment Trend Over Time', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Date', fontsize=14, fontweight='bold')
ax.set_ylabel('Total Enrollments', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.tick_params(labelsize=11)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, '01_monthly_trend.png'), dpi=300, bbox_inches='tight')
print(" Saved: 01_monthly_trend.png")
plt.close()

# Find insights
peak_month = monthly.idxmax()
peak_value = monthly.max()
low_month = monthly.idxmin()
low_value = monthly.min()

print(f"\n INSIGHTS:")
print(f"   Peak month: {peak_month.strftime('%B %Y')} with {peak_value:,.0f} enrollments")
print(f"   Lowest month: {low_month.strftime('%B %Y')} with {low_value:,.0f} enrollments")
print(f"   Variation: {((peak_value - low_value) / low_value * 100):.1f}% difference")

# ==========================================
# ANALYSIS 2: SEASONALITY PATTERN
# ==========================================

print("\n" + "="*70)
print(" ANALYSIS 2: SEASONALITY PATTERN")
print("="*70)

seasonal = df_enrollment.groupby('month')['total_enrollments'].mean()

fig, ax = plt.subplots(figsize=(12, 7))
colors = ['#EE6C4D' if x == seasonal.max() else '#3D5A80' for x in seasonal]
bars = ax.bar(range(1, 13), seasonal.values, color=colors, edgecolor='black', linewidth=1.2)
ax.set_title('Average Enrollments by Month (Seasonality Pattern)', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Month', fontsize=14, fontweight='bold')
ax.set_ylabel('Average Enrollments', fontsize=14, fontweight='bold')
ax.set_xticks(range(1, 13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
ax.grid(True, alpha=0.3, axis='y', linestyle='--')
ax.tick_params(labelsize=11)

# Add value labels on bars
for i, (bar, val) in enumerate(zip(bars, seasonal.values)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:,.0f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, '02_seasonality.png'), dpi=300, bbox_inches='tight')
print(" Saved: 02_seasonality.png")
plt.close()

peak_seasonal_month = seasonal.idxmax()
low_seasonal_month = seasonal.idxmin()
variation_pct = ((seasonal.max() - seasonal.min()) / seasonal.mean() * 100)

print(f"\n📌 INSIGHTS:")
print(f"   Peak enrollment month: Month {peak_seasonal_month} ({peak_seasonal_month})")
print(f"   Lowest enrollment month: Month {low_seasonal_month}")
print(f"   Seasonal variation: {variation_pct:.1f}% from average")

# ==========================================
# ANALYSIS 3: STATE-WISE COMPARISON
# ==========================================

print("\n" + "="*70)
print(" ANALYSIS 3: STATE-WISE COMPARISON")
print("="*70)

state_totals = df_enrollment.groupby('state')['total_enrollments'].sum().sort_values(ascending=False)

# Top 15 states
fig, ax = plt.subplots(figsize=(14, 9))
top15 = state_totals.head(15)
bars = ax.barh(range(len(top15)), top15.values, color='#06A77D', edgecolor='black', linewidth=1)
ax.set_yticks(range(len(top15)))
ax.set_yticklabels(top15.index, fontsize=11)
ax.set_title('Top 15 States by Total Enrollments', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Total Enrollments', fontsize=14, fontweight='bold')
ax.set_ylabel('State', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x', linestyle='--')
ax.invert_yaxis()

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top15.values)):
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2.,
            f'{val:,.0f}', ha='left', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, '03_top_states.png'), dpi=300, bbox_inches='tight')
print(" Saved: 03_top_states.png")
plt.close()

# Bottom 10 states
fig, ax = plt.subplots(figsize=(14, 8))
bottom10 = state_totals.tail(10)
bars = ax.barh(range(len(bottom10)), bottom10.values, color='#D62828', edgecolor='black', linewidth=1)
ax.set_yticks(range(len(bottom10)))
ax.set_yticklabels(bottom10.index, fontsize=11)
ax.set_title('Bottom 10 States - Need Intervention', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Total Enrollments', fontsize=14, fontweight='bold')
ax.set_ylabel('State', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x', linestyle='--')
ax.invert_yaxis()

# Add value labels
for i, (bar, val) in enumerate(zip(bars, bottom10.values)):
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2.,
            f'{val:,.0f}', ha='left', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, '04_bottom_states.png'), dpi=300, bbox_inches='tight')
print(" Saved: 04_bottom_states.png")
plt.close()

print(f"\n INSIGHTS:")
print(f"   Top state: {state_totals.index[0]} with {state_totals.iloc[0]:,.0f} enrollments")
print(f"   Bottom state: {state_totals.index[-1]} with {state_totals.iloc[-1]:,.0f} enrollments")
gap_pct = ((state_totals.iloc[0] - state_totals.iloc[-1]) / state_totals.iloc[0] * 100)
print(f"   Gap: {gap_pct:.1f}% difference between top and bottom states")

# Find underperforming states
avg_enrollments = state_totals.mean()
underperforming = state_totals[state_totals < avg_enrollments * 0.6]
print(f"   Underperforming states (< 60% of average): {len(underperforming)}")

# ==========================================
# ANALYSIS 4: AGE GROUP DISTRIBUTION
# ==========================================

print("\n" + "="*70)
print("👥 ANALYSIS 4: AGE GROUP DISTRIBUTION")
print("="*70)

age_distribution = {
    '0-5 years': df_enrollment['age_0_5'].sum(),
    '5-17 years': df_enrollment['age_5_17'].sum(),
    '18+ years': df_enrollment['age_18_greater'].sum()
}

fig, ax = plt.subplots(figsize=(10, 10))
colors = ['#F4A261', '#E76F51', '#2A9D8F']
wedges, texts, autotexts = ax.pie(age_distribution.values(), 
                                    labels=age_distribution.keys(),
                                    autopct='%1.1f%%',
                                    startangle=90,
                                    colors=colors,
                                    explode=(0.05, 0.05, 0.05),
                                    shadow=True,
                                    textprops={'fontsize': 14, 'fontweight': 'bold'})

ax.set_title('Enrollment Distribution by Age Group', fontsize=18, fontweight='bold', pad=20)

# Add legend with counts
legend_labels = [f'{k}: {v:,.0f}' for k, v in age_distribution.items()]
ax.legend(legend_labels, loc='best', fontsize=11)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, '05_age_distribution.png'), dpi=300, bbox_inches='tight')
print(" Saved: 05_age_distribution.png")
plt.close()

print(f"\n INSIGHTS:")
for age_group, count in age_distribution.items():
    pct = (count / sum(age_distribution.values())) * 100
    print(f"   {age_group}: {count:,.0f} ({pct:.1f}%)")

# ==========================================
# ANALYSIS 5: YEAR-OVER-YEAR GROWTH
# ==========================================

print("\n" + "="*70)
print(" ANALYSIS 5: YEAR-OVER-YEAR GROWTH")
print("="*70)

yearly = df_enrollment.groupby('year')['total_enrollments'].sum()
yoy_growth = yearly.pct_change() * 100

fig, ax = plt.subplots(figsize=(12, 7))
colors = ['#06A77D' if x > 0 else '#D62828' for x in yoy_growth]
bars = ax.bar(yoy_growth.index, yoy_growth.values, color=colors, edgecolor='black', linewidth=1.2, width=0.6)
ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax.set_title('Year-over-Year Growth Rate (%)', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Year', fontsize=14, fontweight='bold')
ax.set_ylabel('Growth Rate (%)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y', linestyle='--')
ax.tick_params(labelsize=11)

# Add value labels
for bar, val in zip(bars, yoy_growth.values):
    if not np.isnan(val):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:+.1f}%', ha='center', 
                va='bottom' if val > 0 else 'top',
                fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, '06_yoy_growth.png'), dpi=300, bbox_inches='tight')
print(" Saved: 06_yoy_growth.png")
plt.close()

print(f"\n INSIGHTS:")
for year, growth in yoy_growth.items():
    if not np.isnan(growth):
        print(f"   {int(year)}: {growth:+.1f}% growth")

# ==========================================
# ANALYSIS 6: PREDICTIVE MODEL
# ==========================================

print("\n" + "="*70)
print(" ANALYSIS 6: PREDICTIVE MODELING")
print("="*70)

# Prepare data for prediction
monthly_data = monthly.reset_index()
monthly_data.columns = ['date', 'enrollments']
monthly_data['month_num'] = range(len(monthly_data))

# Train-test split (80-20)
train_size = int(len(monthly_data) * 0.8)
train = monthly_data[:train_size]
test = monthly_data[train_size:]

# Train model
X_train = train[['month_num']].values
y_train = train['enrollments'].values
X_test = test[['month_num']].values
y_test = test['enrollments'].values

model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Metrics
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"\n MODEL PERFORMANCE:")
print(f"   R² Score: {r2:.3f} ({r2*100:.1f}% variance explained)")
print(f"   Mean Absolute Error: {mae:,.0f} enrollments")
print(f"   MAPE: {mape:.2f}%")

# Forecast 6 months
future_months = np.array(range(len(monthly_data), len(monthly_data) + 6)).reshape(-1, 1)
future_pred = model.predict(future_months)

# Plot
fig, ax = plt.subplots(figsize=(16, 8))
ax.plot(train['date'], train['enrollments'], label='Training Data', 
        marker='o', linewidth=2.5, markersize=5, color='#2A9D8F')
ax.plot(test['date'], test['enrollments'], label='Actual (Test)', 
        marker='o', linewidth=2.5, markersize=6, color='#E76F51')
ax.plot(test['date'], y_pred, label=f'Predicted (R²={r2:.2f})', 
        marker='s', linewidth=2.5, markersize=6, linestyle='--', color='#F4A261')

# Add forecast
last_date = monthly_data['date'].iloc[-1]
future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=6, freq='MS')
ax.plot(future_dates, future_pred, label='6-Month Forecast', 
        marker='^', linewidth=2.5, markersize=7, linestyle=':', color='#9B2226')

ax.set_title('Enrollment Prediction Model with 6-Month Forecast', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Date', fontsize=14, fontweight='bold')
ax.set_ylabel('Total Enrollments', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='best')
ax.grid(True, alpha=0.3, linestyle='--')
ax.tick_params(labelsize=11)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, '07_prediction.png'), dpi=300, bbox_inches='tight')
print(" Saved: 07_prediction.png")
plt.close()

print(f"\n 6-MONTH FORECAST:")
for i, (date, pred) in enumerate(zip(future_dates, future_pred), 1):
    print(f"   Month +{i} ({date.strftime('%b %Y')}): {pred:,.0f} enrollments")

forecast_growth = ((future_pred.sum() / test['enrollments'].tail(6).sum()) - 1) * 100
print(f"   Forecasted growth vs last 6 months: {forecast_growth:+.1f}%")

# ==========================================
# ANALYSIS 7: STATE CLUSTERING
# ==========================================

print("\n" + "="*70)
print(" ANALYSIS 7: STATE CLUSTERING")
print("="*70)

# Prepare state features
state_features = df_enrollment.groupby('state').agg({
    'total_enrollments': ['sum', 'mean', 'std'],
    'age_0_5': 'sum',
    'age_5_17': 'sum',
    'age_18_greater': 'sum'
}).fillna(0)

state_features.columns = ['total', 'avg', 'std', 'age_0_5', 'age_5_17', 'age_18plus']
state_features = state_features.reset_index()

# Normalize
scaler = StandardScaler()
features_to_scale = ['total', 'avg', 'std', 'age_0_5', 'age_5_17', 'age_18plus']
scaled_features = scaler.fit_transform(state_features[features_to_scale])

# Cluster (4 clusters)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
state_features['cluster'] = kmeans.fit_predict(scaled_features)

# Plot
fig, ax = plt.subplots(figsize=(14, 10))
colors_map = {0: '#E76F51', 1: '#2A9D8F', 2: '#F4A261', 3: '#264653'}

for cluster in range(4):
    cluster_data = state_features[state_features['cluster'] == cluster]
    ax.scatter(cluster_data['total'], cluster_data['avg'], 
              label=f'Cluster {cluster} ({len(cluster_data)} states)',
              s=200, alpha=0.7, edgecolors='black', linewidth=1.5,
              color=colors_map[cluster])

ax.set_xlabel('Total Enrollments', fontsize=14, fontweight='bold')
ax.set_ylabel('Average Enrollments', fontsize=14, fontweight='bold')
ax.set_title('State Clustering by Enrollment Patterns', fontsize=18, fontweight='bold', pad=20)
ax.legend(fontsize=12, loc='best')
ax.grid(True, alpha=0.3, linestyle='--')
ax.tick_params(labelsize=11)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, '08_state_clusters.png'), dpi=300, bbox_inches='tight')
print(" Saved: 08_state_clusters.png")
plt.close()

print(f"\n CLUSTER INSIGHTS:")
for cluster in range(4):
    cluster_states = state_features[state_features['cluster'] == cluster]
    print(f"\n   Cluster {cluster}: {len(cluster_states)} states")
    print(f"      States: {', '.join(cluster_states['state'].head(5).tolist())}" + 
          (f", ... (+{len(cluster_states)-5} more)" if len(cluster_states) > 5 else ""))
    print(f"      Avg total enrollments: {cluster_states['total'].mean():,.0f}")
    print(f"      Avg enrollment rate: {cluster_states['avg'].mean():,.0f}")

# ==========================================
# ANALYSIS 8: DEMOGRAPHIC VS ENROLLMENT
# ==========================================

print("\n" + "="*70)
print(" ANALYSIS 8: UPDATE TO ENROLLMENT RATIO")
print("="*70)

# Aggregate by state
enroll_by_state = df_enrollment.groupby('state')['total_enrollments'].sum()
demo_by_state = df_demographic.groupby('state')['total_updates'].sum()

# Merge
update_ratio = pd.DataFrame({
    'enrollments': enroll_by_state,
    'updates': demo_by_state
}).fillna(0)

update_ratio['ratio'] = (update_ratio['updates'] / update_ratio['enrollments'] * 100).replace([np.inf, -np.inf], 0)
update_ratio = update_ratio.sort_values('ratio', ascending=False)

# Plot top 15
fig, ax = plt.subplots(figsize=(14, 9))
top15_ratio = update_ratio.head(15)
bars = ax.barh(range(len(top15_ratio)), top15_ratio['ratio'].values, 
               color='#9B2226', edgecolor='black', linewidth=1)
ax.set_yticks(range(len(top15_ratio)))
ax.set_yticklabels(top15_ratio.index, fontsize=11)
ax.set_title('Top 15 States by Update-to-Enrollment Ratio', fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Update Ratio (%)', fontsize=14, fontweight='bold')
ax.set_ylabel('State', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x', linestyle='--')
ax.invert_yaxis()

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top15_ratio['ratio'].values)):
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2.,
            f'{val:.1f}%', ha='left', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, '09_update_ratio.png'), dpi=300, bbox_inches='tight')
print(" Saved: 09_update_ratio.png")
plt.close()

print(f"\n INSIGHTS:")
print(f"   Avg update ratio: {update_ratio['ratio'].mean():.1f}%")
print(f"   Highest ratio: {update_ratio.index[0]} ({update_ratio['ratio'].iloc[0]:.1f}%)")
high_update_states = update_ratio[update_ratio['ratio'] > update_ratio['ratio'].quantile(0.75)]
print(f"   High update states (>75th percentile): {len(high_update_states)}")
print(f"   These may indicate data quality issues or high migration")

# ==========================================
# FINAL SUMMARY
# ==========================================

print("\n" + "="*70)
print(" "*22 + "ANALYSIS COMPLETE!")
print("="*70)



                    UIDAI HACKATHON
                  COMPLETE ANALYSIS

 LOADING DATA...
 Enrollment: 1,006,029 rows
 Demographic: 2,071,700 rows
 Biometric: 1,861,108 rows

🧹 CLEANING DATA...
 Cleaned enrollment: 319,718 rows
 Date range: 2025-01-04 00:00:00 to 2025-12-11 00:00:00
 Total enrollments: 2,622,225
 States: 49
 Districts: 954

 ANALYSIS 1: TIME SERIES TREND
 Saved: 01_monthly_trend.png

 INSIGHTS:
   Peak month: January 2025 with 1,366,494 enrollments
   Lowest month: July 2025 with 56,194 enrollments
   Variation: 2331.7% difference

 ANALYSIS 2: SEASONALITY PATTERN
 Saved: 02_seasonality.png

📌 INSIGHTS:
   Peak enrollment month: Month 1 (1)
   Lowest enrollment month: Month 7
   Seasonal variation: 577.3% from average

 ANALYSIS 3: STATE-WISE COMPARISON
 Saved: 03_top_states.png
 Saved: 04_bottom_states.png

 INSIGHTS:
   Top state: Uttar Pradesh with 554,062 enrollments
   Bottom state: Westbengal with 3 enrollments
   Gap: 100.0% difference between top and bottom sta